# F.D.A.C. MiniProject2

## Scraping SourceForge

### Load libraries and define db

In [1]:
import sys
import pymongo
import json
import time
import datetime
from bs4 import BeautifulSoup
import requests as req
import re

dbname = "fdac18mp2" #please use this database
collname = "glprj_cjohn3" #please modify so you store data in your collection
# beginning page index
begin = "0"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]

### Collect project names

In [3]:
# Define number of search pages to iterate
PAGES= 20

# Collect URLs
url=[]
for i in range(1,PAGES):
    url.append("https://sourceforge.net/directory/os%3Amac/?q=j&page=" + str(i))
    
# Get URL responses
response= [req.get(link) for link in url]

In [4]:
# Function to select only unique entries in list
def unique(seq): 
   checked = []
   for e in seq:
       if e not in checked:
           checked.append(e)
   return checked

In [5]:
# Iterate through responses and extract project names
j_proj=[]
for resp in response:
    # Parse web response
    html_soup= BeautifulSoup(resp.text, 'html.parser')
    soup_string= str(html_soup)
    
    # Match all projects
    regex = "(?<=\/projects\/).+?(?=\/)"
    matches = re.findall(regex, soup_string)
    j_matches= [i for i in matches if i.startswith('j')]
    j_proj.append(j_matches)

### Check project status and save link

In [46]:
# Make list of unique project names
j_list = unique([item for sublist in j_proj for item in sublist])

# Iterate through projects, check status and save link
sf_api= "https://sourceforge.net/rest/p/"

projects=[]
count=0

while count<50:
    for proj in j_list:
        resp= req.get(sf_api+proj)
        if(resp.status_code == 404):
            continue
            
        text= json.loads(resp.text)
        if(text['status']=='active'):
            coll.insert_one(text)            
            count= count+1


### Check entries exist

In [48]:
for proj in coll.find({}):
    print(proj)
    print('\n\n\n')

{'preferred_support_tool': '', 'socialnetworks': [], 'categories': {'developmentstatus': [], 'environment': [], 'audience': [], 'database': [], 'license': [], 'translation': [], 'topic': [], 'language': [], 'os': []}, '_id': '592f7b1d3bfd8160daaf9be3', 'status': 'active', 'moved_to_url': '', 'video_url': '', 'preferred_support_url': '', 'private': False, 'url': 'https://sourceforge.net/p/j-eoq-sa/', 'creation_date': '2017-06-01', 'shortname': 'j-eoq-sa', 'developers': [{'username': 'valdecypereira', 'url': 'https://sourceforge.net/u/valdecypereira/', 'name': 'Valdecy Pereira'}], 'external_homepage': 'https://j-eoq-sa.sourceforge.io', 'summary': '', 'name': 'J-EOQ-SA', 'icon_url': None, 'tools': [{'mount_point': 'code', 'icons': {'32': 'images/code_32.png', '48': 'images/code_48.png', '24': 'images/code_24.png'}, 'mount_label': 'Code', 'tool_label': 'Git', 'name': 'git', 'installable': True, 'url': '/p/j-eoq-sa/code/'}, {'mount_point': 'tickets', 'icons': {'32': 'images/tickets_32.png',

## Scraping GitLab

In [49]:
import requests 

letter= 'j'

beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"


gleft = 10

header = {'per_page': 99}

# Check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

# Send queries and extract urls 
def get(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    count = 0

    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            for el in array:
                if el['name'].lower().startswith(letter):
                    el['site'] = "git"
                    count += 1
                    coll.insert_one(el)
                    if count > 49:
                        return
                
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            if el['name'].lower().startswith(letter):
                                el['site'] = "git"
                                count += 1
                                coll.insert_one(el)
                                if count > 49:
                                    return
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')

In [50]:
# Start retrieving        
get(beginurl,coll)